# 04: Channel Effects

This notebook explores how wireless channels affect transmitted signals.

## Learning Objectives
- Understand AWGN and its impact on constellation
- Explore Rayleigh and Rician fading
- Analyze multipath propagation
- Study Doppler effects from mobility

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from r4w_python import plot_constellation, plot_time_domain

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## Additive White Gaussian Noise (AWGN)

AWGN is the fundamental noise model:
- **Additive**: Adds to signal
- **White**: Uniform power across all frequencies
- **Gaussian**: Amplitude follows normal distribution

$$y = x + n, \quad n \sim \mathcal{N}(0, \sigma^2)$$

In [ ]:
def awgn_channel(signal, snr_db):
    """Add AWGN at specified SNR."""
    signal_power = np.mean(np.abs(signal)**2)
    noise_power = signal_power / (10**(snr_db/10))
    noise = np.sqrt(noise_power/2) * (np.random.randn(len(signal)) + 
                                       1j * np.random.randn(len(signal)))
    return signal + noise

# Generate QPSK symbols
qpsk = np.exp(1j * np.pi/4 * (2 * np.arange(4) + 1))
np.random.seed(42)
tx = qpsk[np.random.randint(0, 4, 500)]

In [ ]:
# Compare different SNR levels
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
snrs = [0, 5, 10, 20]

for ax, snr in zip(axes.flat, snrs):
    rx = awgn_channel(tx, snr)
    plot_constellation(rx, title=f'QPSK @ {snr} dB SNR', ax=ax)

plt.tight_layout()
plt.show()

## Fading Channels

### Rayleigh Fading
When there's no line-of-sight (NLOS) path, the channel gain is Rayleigh distributed:

$$h \sim \mathcal{CN}(0, 1)$$

### Rician Fading
With a line-of-sight (LOS) component, we get Rician fading:

$$h = \sqrt{\frac{K}{K+1}} + \sqrt{\frac{1}{K+1}} \cdot \mathcal{CN}(0, 1)$$

K-factor: ratio of LOS power to scattered power

In [ ]:
def rayleigh_channel(signal, snr_db):
    """Apply Rayleigh fading then AWGN."""
    # Rayleigh gain (single tap, flat fading)
    h = (np.random.randn(len(signal)) + 1j * np.random.randn(len(signal))) / np.sqrt(2)
    faded = signal * h
    return awgn_channel(faded, snr_db)

def rician_channel(signal, snr_db, k_factor):
    """Apply Rician fading then AWGN."""
    los = np.sqrt(k_factor / (k_factor + 1))
    nlos = np.sqrt(1 / (k_factor + 1))
    scatter = (np.random.randn(len(signal)) + 1j * np.random.randn(len(signal))) / np.sqrt(2)
    h = los + nlos * scatter
    faded = signal * h
    return awgn_channel(faded, snr_db)

In [ ]:
# Compare fading types
snr = 20
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

channels = [
    ('AWGN Only', awgn_channel(tx, snr)),
    ('Rayleigh Fading', rayleigh_channel(tx, snr)),
    ('Rician K=1', rician_channel(tx, snr, 1)),
    ('Rician K=10', rician_channel(tx, snr, 10)),
]

for ax, (name, rx) in zip(axes.flat, channels):
    plot_constellation(rx, title=f'{name} @ {snr} dB', ax=ax)

plt.tight_layout()
plt.show()

## Multipath Propagation

Signals can take multiple paths, each with different:
- **Delay**: Path length difference
- **Attenuation**: Path loss
- **Phase shift**: Due to reflections

This causes frequency-selective fading.

In [ ]:
def multipath_channel(signal, sample_rate, taps):
    """Apply multipath channel.
    
    Args:
        signal: Input signal
        sample_rate: Sample rate in Hz
        taps: List of (delay_sec, gain_linear, phase_rad)
    """
    output = np.zeros(len(signal) + int(max(t[0] for t in taps) * sample_rate) + 1, 
                      dtype=complex)
    
    for delay, gain, phase in taps:
        delay_samples = int(delay * sample_rate)
        output[delay_samples:delay_samples+len(signal)] += signal * gain * np.exp(1j * phase)
    
    return output[:len(signal)]

# Example: 3-tap channel
taps = [
    (0.0, 1.0, 0.0),           # Direct path
    (0.00001, 0.5, np.pi/3),   # Reflection 1
    (0.00003, 0.3, np.pi),     # Reflection 2
]

In [ ]:
# Generate a test signal and apply multipath
sample_rate = 100000
t = np.arange(0, 0.001, 1/sample_rate)
pulse = np.exp(2j * np.pi * 10000 * t) * np.hanning(len(t))

# Apply channel
rx = multipath_channel(pulse, sample_rate, taps)

fig, axes = plt.subplots(2, 1, figsize=(12, 6))

axes[0].plot(t * 1000, pulse.real, label='TX')
axes[0].plot(t * 1000, rx[:len(t)].real, label='RX (Multipath)', alpha=0.7)
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Multipath Effect on Pulse')
axes[0].legend()

# Frequency response
freq_resp = np.fft.fftshift(np.fft.fft(rx, 1024))
freqs = np.linspace(-sample_rate/2, sample_rate/2, 1024)
axes[1].plot(freqs/1000, 20*np.log10(np.abs(freq_resp) + 1e-10))
axes[1].set_xlabel('Frequency (kHz)')
axes[1].set_ylabel('Gain (dB)')
axes[1].set_title('Frequency-Selective Fading')

plt.tight_layout()
plt.show()

## Doppler Effect

When transmitter/receiver are moving, the received frequency shifts:

$$f_d = \frac{v}{c} \cdot f_c \cdot \cos(\theta)$$

At 900 MHz carrier:
- Walking (5 km/h): ~4 Hz Doppler
- Driving (60 km/h): ~50 Hz Doppler
- High-speed train (300 km/h): ~250 Hz Doppler

In [ ]:
def doppler_shift(signal, sample_rate, doppler_hz):
    """Apply frequency shift due to Doppler."""
    t = np.arange(len(signal)) / sample_rate
    return signal * np.exp(2j * np.pi * doppler_hz * t)

# Generate a tone
sample_rate = 48000
t = np.arange(0, 0.1, 1/sample_rate)
carrier_freq = 1000  # Hz (for visualization)
signal = np.exp(2j * np.pi * carrier_freq * t)

# Apply different Doppler shifts
doppler_values = [0, 10, 50, 100]  # Hz

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

for ax, fd in zip(axes.flat, doppler_values):
    shifted = doppler_shift(signal, sample_rate, fd)
    
    # Compute spectrum
    spectrum = np.fft.fftshift(np.fft.fft(shifted, 4096))
    freqs = np.linspace(-sample_rate/2, sample_rate/2, 4096)
    power = 20 * np.log10(np.abs(spectrum) + 1e-10)
    
    ax.plot(freqs, power)
    ax.axvline(carrier_freq + fd, color='r', linestyle='--', label=f'Peak @ {carrier_freq+fd} Hz')
    ax.set_xlim(800, 1200)
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Power (dB)')
    ax.set_title(f'Doppler Shift = {fd} Hz')
    ax.legend()

plt.tight_layout()
plt.show()

## Exercises

1. **SNR threshold**: At what SNR can you no longer distinguish QPSK symbols reliably?

2. **Rician K-factor**: At what K-factor does Rician fading become essentially AWGN?

3. **Doppler at different frequencies**: Calculate Doppler shift for a car at 100 km/h at:
   - 900 MHz (GSM)
   - 2.4 GHz (WiFi)
   - 28 GHz (5G mmWave)

In [ ]:
# Your code here
